# Второе и третье задание 

Логистическая регрессия - тренирую на датасете с новыми фичами; CV-score = 0.7296 LB-score = 0.666

Беггинг - на датасете без новых фич CV-score = 0.7257 LB-score = 0.673



In [1]:

import pandas as pd
import numpy as np




train = pd.read_csv('train.csv', sep = '\t', index_col = 'Unnamed: 0')
test = pd.read_csv('test.csv', sep = '\t', index_col = 'Unnamed: 0')

train.drop(['140', '152', '160', '164','305', '303', '301', '193', '192', '186', '150', '141', '138', '130'],inplace = True,axis = 1)
test.drop(['140', '152', '160', '164','305', '303', '301', '193', '192', '186', '150', '141', '138', '130'],inplace = True, axis = 1)

X_train, y_train = train[train.columns[1:]], train['0']
X_test, y_test = test[test.columns[1:]], test['0'].fillna(0, inplace = True)



# matthews correlation of binary columns with target variable
binary_list = [i for i in X_train.columns if len(X_train[i].value_counts()) == 2]

from sklearn.metrics import matthews_corrcoef
import operator

matt_corr = {}
high_negative = {}
high_positive = {}

for i in binary_list:
    matt_corr[i] = matthews_corrcoef(y_train.values, X_train[i].values)
    if matt_corr[i] >= 0.07:
        high_positive[i] = matthews_corrcoef(y_train.values, X_train[i].values)
    elif matt_corr[i] <= -0.07:
        high_negative[i] = matthews_corrcoef(y_train.values, X_train[i].values)
        
        
    
high_positive_sorted = sorted(high_positive.items(), key=operator.itemgetter(1))
high_negative_sorted = sorted(high_negative.items(), key=operator.itemgetter(1))    
matt_corr_sorted = sorted(matt_corr.items(), key=operator.itemgetter(1))

for i in range(len(list(high_positive.keys()))):
    for j in range(i+1 ,len(list(high_positive.keys()))):
        if matthews_corrcoef(X_train[list(high_positive.keys())[i]], X_train[list(high_positive.keys())[j]]) < 0.3:
            mix = np.logical_or(X_train[list(high_positive.keys())[i]], X_train[list(high_positive.keys())[j]])
            mix = np.array([int(i) for i in mix])
            if (matthews_corrcoef(mix, X_train[list(high_positive.keys())[i]]) < 0.99) and (matthews_corrcoef(mix, X_train[list(high_positive.keys())[i]])) < 0.99:
                X_train['mixOR '+list(high_positive.keys())[i]+'-'+list(high_positive.keys())[j]] = mix
                mix_1 = np.logical_or(X_test[list(high_positive.keys())[i]], X_test[list(high_positive.keys())[j]])
                mix_1 = np.array([int(i) for i in mix_1])                
                X_test['mixOR '+list(high_positive.keys())[i]+'-'+list(high_positive.keys())[j]] = mix_1


        if matthews_corrcoef(X_train[list(high_positive.keys())[i]], X_train[list(high_positive.keys())[j]]) > 0.6:
            mix = np.logical_and(X_train[list(high_positive.keys())[i]], X_train[list(high_positive.keys())[j]])
            mix = np.array([int(i) for i in mix])
            if (matthews_corrcoef(mix, X_train[list(high_positive.keys())[i]]) < 0.99) and (matthews_corrcoef(mix, X_train[list(high_positive.keys())[i]])) < 0.99:
                X_train['mixAND '+list(high_positive.keys())[i]+'-'+ list(high_positive.keys())[j]] = mix
                mix_1 = np.logical_and(X_test[list(high_positive.keys())[i]], X_test[list(high_positive.keys())[j]])
                mix_1 = np.array([int(i) for i in mix_1])      
                X_test['mixAND '+list(high_positive.keys())[i]+'-'+ list(high_positive.keys())[j]] = mix_1



for i in range(len(list(high_negative.keys()))):
    for j in range(i+1 ,len(list(high_negative.keys()))):        
        if matthews_corrcoef(X_train[list(high_negative.keys())[i]], X_train[list(high_negative.keys())[j]]) < 0.3:
            mix = np.logical_or(X_train[list(high_negative.keys())[i]], X_train[list(high_negative.keys())[j]])
            mix = np.array([int(i) for i in mix])
            if (matthews_corrcoef(mix, X_train[list(high_negative.keys())[i]]) < 0.99) and (matthews_corrcoef(mix, X_train[list(high_negative.keys())[i]])) < 0.99:
                X_train['mixOR '+list(high_negative.keys())[i]+'-'+list(high_negative.keys())[j]] = mix
                mix_1 = np.logical_or(X_test[list(high_negative.keys())[i]], X_test[list(high_negative.keys())[j]])
                mix_1 = np.array([int(i) for i in mix_1])                
                X_test['mixOR '+list(high_negative.keys())[i]+'-'+list(high_negative.keys())[j]] = mix_1


                
        if matthews_corrcoef(X_train[list(high_negative.keys())[i]], X_train[list(high_negative.keys())[j]]) > 0.6:
            mix = np.logical_and(X_train[list(high_negative.keys())[i]], X_train[list(high_negative.keys())[j]])
            mix = np.array([int(i) for i in mix])
            if (matthews_corrcoef(mix, X_train[list(high_negative.keys())[i]]) < 0.99) and (matthews_corrcoef(mix, X_train[list(high_negative.keys())[i]])) < 0.99:
                X_train['mixAND '+list(high_negative.keys())[i]+'-'+ list(high_negative.keys())[j]] = mix
                
                mix_1 = np.logical_and(X_test[list(high_negative.keys())[i]], X_test[list(high_negative.keys())[j]])
                mix_1 = np.array([int(i) for i in mix_1])                
                X_test['mixOR '+list(high_negative.keys())[i]+'-'+list(high_negative.keys())[j]] = mix_1


X_train_old, y_train_old = train[train.columns[1:]], train['0']
X_test_old, y_test_old = test[test.columns[1:]], test['0'].fillna(0, inplace = True)



In [9]:
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression


skf = StratifiedKFold(n_splits=4)
classifier = LogisticRegression(max_iter = 100)
pipeline = Pipeline([('classifier', classifier)])

param_grid = [{'classifier__C':[0.005, 0.04588385, 0.05, 0.1, 0.15, 0.2 , 0.3, 0.8, 1, 4, 7, 10], 'classifier__class_weight':[None, 'balanced']}]
grid_search = GridSearchCV(pipeline, param_grid = param_grid, cv = skf, scoring = 'roc_auc', n_jobs = 3, verbose = 2)




In [ ]:
grid_search.fit(X_train,y_train)

In [11]:
print(grid_search.best_params_)
print(grid_search.best_score_)

{'classifier__C': 0.04588385, 'classifier__class_weight': 'balanced'}
0.729693078495


In [12]:
best_lr = grid_search.best_estimator_
best_lr.fit(X_train,y_train)

Pipeline(steps=[('classifier', LogisticRegression(C=0.04588385, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False))])

In [12]:
best_lr = grid_search.best_estimator_
best_lr.fit(X_train,y_train)

Pipeline(steps=[('classifier', LogisticRegression(C=0.04588385, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False))])

In [24]:
y_test = best_lr.predict(X_test)

test['0'] = y_test
header = ['_VAL_']
test['0'].to_csv('lr_2.csv',sep =',', header = header, index_label = ['_ID_']) 

# Бэггинг

In [3]:
## fitting on data without any feature generation
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier


skf = StratifiedKFold(shuffle = True, random_state = 42)

lg = LogisticRegression(class_weight= 'balanced')
parameters = {'max_features': [0.95, 1.], 'max_samples': [0.85, 0.9, 0.95], "base_estimator__C": np.linspace(0.005,0.1,7)}
bg = BaggingClassifier(lg, random_state=42, n_estimators=5)

grid_search_1 = GridSearchCV(bg, parameters, scoring ='roc_auc', cv = skf, verbose = 2, n_jobs = 3)
grid_search_1 = grid_search_1.fit(X_train_old, y_train_old)

print(grid_search_1.best_score_)




Fitting 3 folds for each of 42 candidates, totalling 126 fits
[CV] max_samples=0.85, max_features=0.95, base_estimator__C=0.005 ....
[CV] max_samples=0.85, max_features=0.95, base_estimator__C=0.005 ....
[CV] max_samples=0.85, max_features=0.95, base_estimator__C=0.005 ....
[CV]  max_samples=0.85, max_features=0.95, base_estimator__C=0.005, total=   4.2s
[CV] max_samples=0.9, max_features=0.95, base_estimator__C=0.005 .....
[CV]  max_samples=0.85, max_features=0.95, base_estimator__C=0.005, total=   4.4s
[CV] max_samples=0.9, max_features=0.95, base_estimator__C=0.005 .....
[CV]  max_samples=0.85, max_features=0.95, base_estimator__C=0.005, total=   4.5s
[CV] max_samples=0.9, max_features=0.95, base_estimator__C=0.005 .....
[CV]  max_samples=0.9, max_features=0.95, base_estimator__C=0.005, total=   4.5s
[CV] max_samples=0.95, max_features=0.95, base_estimator__C=0.005 ....
[CV]  max_samples=0.9, max_features=0.95, base_estimator__C=0.005, total=   4.4s
[CV] max_samples=0.95, max_featur

[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:  1.1min


[CV] max_samples=0.85, max_features=0.95, base_estimator__C=0.0366666666667 
[CV]  max_samples=0.85, max_features=0.95, base_estimator__C=0.0366666666667, total=   6.8s
[CV] max_samples=0.9, max_features=0.95, base_estimator__C=0.0366666666667 
[CV]  max_samples=0.85, max_features=0.95, base_estimator__C=0.0366666666667, total=   6.5s
[CV] max_samples=0.9, max_features=0.95, base_estimator__C=0.0366666666667 
[CV]  max_samples=0.85, max_features=0.95, base_estimator__C=0.0366666666667, total=   6.8s
[CV] max_samples=0.9, max_features=0.95, base_estimator__C=0.0366666666667 
[CV]  max_samples=0.9, max_features=0.95, base_estimator__C=0.0366666666667, total=   6.7s
[CV] max_samples=0.95, max_features=0.95, base_estimator__C=0.0366666666667 
[CV]  max_samples=0.9, max_features=0.95, base_estimator__C=0.0366666666667, total=   6.7s
[CV] max_samples=0.95, max_features=0.95, base_estimator__C=0.0366666666667 
[CV]  max_samples=0.9, max_features=0.95, base_estimator__C=0.0366666666667, total=

[CV] max_samples=0.95, max_features=1.0, base_estimator__C=0.0683333333333 
[CV]  max_samples=0.95, max_features=1.0, base_estimator__C=0.0683333333333, total=  11.0s
[CV] max_samples=0.85, max_features=0.95, base_estimator__C=0.0841666666667 
[CV]  max_samples=0.95, max_features=1.0, base_estimator__C=0.0683333333333, total=  11.1s
[CV] max_samples=0.85, max_features=0.95, base_estimator__C=0.0841666666667 
[CV]  max_samples=0.95, max_features=1.0, base_estimator__C=0.0683333333333, total=  11.3s
[CV] max_samples=0.85, max_features=0.95, base_estimator__C=0.0841666666667 
[CV]  max_samples=0.85, max_features=0.95, base_estimator__C=0.0841666666667, total=   9.8s
[CV] max_samples=0.9, max_features=0.95, base_estimator__C=0.0841666666667 
[CV]  max_samples=0.85, max_features=0.95, base_estimator__C=0.0841666666667, total=   9.3s
[CV] max_samples=0.9, max_features=0.95, base_estimator__C=0.0841666666667 
[CV]  max_samples=0.85, max_features=0.95, base_estimator__C=0.0841666666667, total=

[Parallel(n_jobs=3)]: Done 126 out of 126 | elapsed:  5.4min finished


0.725763356832


In [4]:
print(grid_search_1.best_params_)

best_bag = grid_search_1.best_estimator_
best_bag.fit(X_train_old, y_train_old)

y_test = best_bag.predict(X_test_old)
test['0'] = y_test



{'max_samples': 0.9, 'max_features': 1.0, 'base_estimator__C': 0.052499999999999998}


8055148

In [15]:
header = ['_VAL_']
test['0'].to_csv('bag_2.csv',sep =',', header = header, index_label = ['_ID_']) 